## 載入模型

In [7]:
label_to_coordinates = {
    "1-1": (0, 0), "1-2": (0.6, 0), "1-3": (1.2, 0), "1-4": (1.8, 0), "1-5": (2.4, 0), "1-6": (3.0, 0),"1-7": (3.6, 0), "1-8": (4.2, 0), "1-9": (4.8, 0), "1-10": (5.4, 0), "1-11": (6.0, 0),
    "2-1": (0, 0.6), "2-11": (6.0, 0.6),
    "3-1": (0, 1.2), "3-11": (6.0, 1.2),
    "4-1": (0, 1.8), "4-11": (6.0, 1.8),
    "5-1": (0, 2.4), "5-11": (6.0, 2.4),
    "6-1": (0, 3.0), "6-2": (0.6, 3.0), "6-3": (1.2, 3.0), "6-4": (1.8, 3.0), "6-5": (2.4, 3.0),"6-6": (3.0, 3.0), "6-7": (3.6, 3.0), "6-8": (4.2, 3.0), "6-9": (4.8, 3.0), "6-10": (5.4, 3.0), "6-11": (6.0, 3.0),
    "7-1": (0, 3.6), "7-11": (6.0, 3.6),
    "8-1": (0, 4.2), "8-11": (6.0, 4.2),
    "9-1": (0, 4.8), "9-11": (6.0, 4.8),
    "10-1": (0, 5.4), "10-11": (6.0, 5.4),
    "11-1": (0, 6.0), "11-2": (0.6, 6.0), "11-3": (1.2, 6.0), "11-4": (1.8, 6.0), "11-5": (2.4, 6.0),"11-6": (3.0, 6.0), "11-7": (3.6, 6.0), "11-8": (4.2, 6.0), "11-9": (4.8, 6.0), "11-10": (5.4, 6.0), "11-11": (6.0, 6.0)
}
label_mapping = {
    '11': '1-1','10': '1-2','9': '1-3','8': '1-4','7': '1-5','6': '1-6','5': '1-7','4': '1-8','3': '1-9','2': '1-10','1': '1-11',
    '12': '2-1','30': '2-11',
    '13': '3-1','29': '3-11',
    '14': '4-1','28': '4-11',
    '15': '5-1','27': '5-11',
    '16': '6-1','17': '6-2','18': '6-3','19': '6-4','20': '6-5','21': '6-6','22': '6-7','23': '6-8','24': '6-9','25': '6-10','26': '6-11',
    '49': '7-1','31': '7-11',
    '48': '8-1','32': '8-11',
    '47': '9-1','33': '9-11',
    '46': '10-1','34': '10-11',
    '45': '11-1','44': '11-2','43': '11-3','42': '11-4','41': '11-5','40': '11-6','39': '11-7','38': '11-8','37': '11-9','36': '11-10','35': '11-11'
}

In [8]:
from tensorflow.keras.models import load_model
from tensorflow import keras
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib  # 用於保存模型
import json
from sklearn.model_selection import train_test_split

In [ ]:
selected_columns = ['Label',
                        'AP1_Distance (mm)','AP4_Distance (mm)',
                        'AP1_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

X_testing_selected_columns = [
                'AP1_Distance (mm)','AP4_Distance (mm)',
                        'AP1_StdDev (mm)','AP4_StdDev (mm)',
                                'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi']  

label_column = 'Label'
target_column = 'Label'


#  'AP1_Rssi','AP2_Rssi','AP3_Rssi','AP4_Rssi'
# 'AP1_Distance (mm)','AP2_Distance (mm)','AP3_Distance (mm)','AP4_Distance (mm)',
# 'AP1_StdDev (mm)','AP2_StdDev (mm)','AP3_StdDev (mm)','AP4_StdDev (mm)',

In [ ]:
ap = '2mcAP_Best'
root = '80 data per RP transfer revised'

dataamount = 80
N_val = 16

N_train = dataamount # 訓練集每個類別至少要有 N_train 筆資料
test_val_ratio = 1  # 剩餘資料中，50% 作為驗證集，50% 作為測試集

weekrepresent = [['0','0'],['0','1'],['1','2'],['2','3'],['3','4'],['4','5']]
alldatadate = ['2024_12_21','2024_12_27','2025_01_03','2025_01_10','2025_02_28']
number_of_week = len(alldatadate)

ALL_trainingtime = []
ALL_loss = []
ALL_accuracy = []
ALL_mean_mde = []

In [11]:
for i in range(number_of_week):
    
    # 載入模型與標準化器
    base_model = load_model(f'{root}/DNN_best_model{ap}_{weekrepresent[i][0]}week_to_{weekrepresent[i][1]}week_{dataamount}dataPerRP.h5')
    scaler = joblib.load(f'{root}/scaler_{ap}.pkl')

    # 讀取測試資料 2024_12_21   2024_12_27   2025_01_03   2025_01_10   2025_02_28
    test_file_path = f"timestamp_allignment_Balanced_{alldatadate[i]}_rtt_logs.csv"  # 測試資料的檔案名稱
    date_test = f"{alldatadate[i]}"
    modelname = f"DNN {ap}s BEST_{dataamount}data_{alldatadate[i]}"
    test_data = pd.read_csv(test_file_path, usecols=selected_columns)
    # test_data
    

    # 凍結所有層
    for layer in base_model.layers[:-1]:  # 除了最後一層 (Output Layer)
        layer.trainable = False

    # 確認哪些層可訓練
    base_model.summary()

    base_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])




    # 資料前處理 (一): 刪除前後n筆資料
    n = 10
    # 確保依據Label排序
    test_data = test_data.sort_values(by=label_column).reset_index(drop=True)

    # 建立一個空的 DataFrame 用於存放處理後的資料
    test_processed_data = pd.DataFrame(columns=test_data.columns)

    # 針對每個Label群組進行處理
    for label, group in test_data.groupby(label_column):
        # 刪除前n筆和後n筆資料
        if len(group) > 2 * n:  # 確保群組資料足夠
            group = group.iloc[n:-n]
        else:
            group = pd.DataFrame()  # 若資料不足，刪除整個群組
        # 將處理後的群組資料加入
        test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)

    # test_processed_data
    # Calculate the number of rows with NaN values
    nan_rows = test_processed_data.isnull().any(axis=1).sum()

    # Print the result
    # print(f"Number of rows with NaN values: {nan_rows}")

    # 找出包含 NaN 的列
    rows_with_nan = test_processed_data[test_processed_data.isnull().any(axis=1)]

    # # 印出這些列
    # print("Rows with NaN values:")
    # print(rows_with_nan)
    test_data_imputed = test_processed_data.groupby(label_column).apply(
        lambda group: group.fillna(group.mean())
    ).reset_index()

    # Calculate the number of rows with NaN values
    nan_rows = test_data_imputed.isnull().any(axis=1).sum()

    # Print the result
    # print(f"Number of rows with NaN values: {nan_rows}")

    # 找出包含 NaN 的列
    rows_with_nan = test_data_imputed[test_data_imputed.isnull().any(axis=1)]

    test_data_imputed

    reverse_label_mapping = {v: int(k) - 1 for k, v in label_mapping.items()}  # 讓數字標籤 -1

    # 建立 Label 映射
    y_test = test_data_imputed[target_column]
    y_test_numeric = y_test.map(reverse_label_mapping)

    # print("Final reverse_label_mapping in DNN:", reverse_label_mapping)
    # print("y_numeric unique values in DNN:", y_test_numeric.unique())

    y_test_numeric

    # 把label部分拿掉
    X_test = test_data_imputed.drop(columns=['level_1','Label'])
    print(X_test.head())

    # 確保測試資料的特徵與訓練資料的特徵一致
    X_test = X_test[X_testing_selected_columns]  # 選取相同的特徵

    # print(type(X_test))

    # 使用之前訓練時的標準化器 (scaler) 來標準化測試數據
    X_test_scaled = scaler.transform(X_test)

    # 會發現如果用 train_test_split的方法會有資料分布不平均問題，解決辦法如下
    from sklearn.model_selection import StratifiedShuffleSplit


    # 轉為 DataFrame 方便操作
    data = pd.DataFrame(X_test_scaled)
    data['label'] = y_test_numeric  # 加入 label 欄位

    # print((data['label'] == 10).sum())  # 直接計算 True 的數量

    # 轉為 DataFrame 方便操作
    data = pd.DataFrame(X_test_scaled)
    data['label'] = y_test_numeric  # 加入 label 欄位

    # data
    # 儲存訓練集（但這時包含 validation 的部分）
    # train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42)
    # 儲存訓練集（確保每個類別只選 1 筆，若某類少於 N_train，則取全部）
    # train_data_full = data.groupby('label', group_keys=False).apply(lambda x: x.sample(n=min(N_train, len(x)), replace=False, random_state=42)).reset_index(drop=True)
    train_data_full = data.groupby('label', group_keys=False).sample(n=N_train, replace=False, random_state=42) # 
    # 確保 `train_data_full` 內部的 `label` 數量正確
    # print(train_data_full['label'].value_counts())  # 每個類別應該剛好 1 筆
    # train_data_full


    if N_val > 0:
        sss = StratifiedShuffleSplit(n_splits=1, test_size=N_val / N_train, random_state=42) # 
        train_index, val_index = next(sss.split(train_data_full.drop(columns=['label']), train_data_full['label']))
        train_data = train_data_full.iloc[train_index]
        val_data = train_data_full.iloc[val_index]
        
    else:
        val_data = pd.DataFrame(columns=data.columns)  # 若沒有 validation data，建立空 DataFrame\
        train_data = train_data_full

    # 剩下的資料（未被抽入 train_data_full 的部分）直接作為 test set
    remaining_data = data.drop(train_data_full.index)
    # print(len(remaining_data))


    # **轉換為 NumPy 陣列**
    X_train, y_train = train_data.drop(columns=['label']).values, train_data['label'].values
    X_val, y_val = val_data.drop(columns=['label']).values, val_data['label'].values
    X_test, y_test = remaining_data.drop(columns=['label']).values, remaining_data['label'].values

    # **確認數據切分結果**
    # print(f"Training set: {len(X_train)} samples, {len(np.unique(y_train))} unique labels")
    # print(f"Validation set: {len(X_val)} samples, {len(np.unique(y_val))} unique labels")
    # print(f"Test set: {len(X_test)} samples, {len(np.unique(y_test))} unique labels")

   # **計算每個 Set 內各 Label 的資料數量**
    train_label_counts = pd.Series(y_train).value_counts().sort_index()
    val_label_counts = pd.Series(y_val).value_counts().sort_index()
    test_label_counts = pd.Series(y_test).value_counts().sort_index()

    # **確保所有 Labels 都有出現在三個 Set 裡**
    all_labels = sorted(set(train_label_counts.index) | set(val_label_counts.index) | set(test_label_counts.index))
    label_distribution = pd.DataFrame(index=all_labels)

    label_distribution["Training Set"] = train_label_counts
    label_distribution["Validation Set"] = val_label_counts
    label_distribution["Test Set"] = test_label_counts

    # **用 0 填補缺失值（表示該 Label 在該 Set 中沒有數據）**
    label_distribution = label_distribution.fillna(0).astype(int)

    from IPython.display import display
    display(label_distribution)

    import time
    # 記錄開始時間
    start_time = time.time()


    from tensorflow.keras.callbacks import EarlyStopping

    if N_val > 0:
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    else:
        early_stop = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)


    # 確保變數命名一致
    X_train_small = X_train  # 確保這裡用的變數和前面一致
    y_train_small = y_train

    # 設定 batch_size
    batch_size = min(32, max(8, len(X_train_small) // 2))  # 避免 batch size 過大
    # batch_size = 32
    print(batch_size)

    if N_val > 0:
        base_model.fit(X_train_small, y_train_small,validation_data=(X_val, y_val), epochs=10000, batch_size=batch_size, callbacks=[early_stop])
    else:
        base_model.fit(X_train_small, y_train_small, epochs=10000, batch_size=batch_size, callbacks=[early_stop])


    # 記錄結束時間
    end_time = time.time()


    # Needsave
    # 計算訓練時間（秒）
    training_time = end_time - start_time
    print(f"訓練時間：{training_time:.2f} 秒")
    ALL_trainingtime.append(training_time)

    # Needsave
    loss, accuracy = base_model.evaluate(X_test, y_test)
    print(f"Evaluation on 90% unused data - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    ALL_loss.append(loss)
    ALL_accuracy.append(accuracy)

    # 預測測試資料
    y_test_pred_numeric = base_model.predict(X_test_scaled)
    y_pred_classes = np.argmax(y_test_pred_numeric, axis=1)

    # 轉換為原本的 Label
    y_test_pred_labels = [label_mapping[str(num + 1)] for num in y_pred_classes]  # 補回 +1
    y_test_pred_labels

    # 讀取測試資料的實際 Label
    y_test_actual = test_data_imputed[target_column]
    test_data_imputed

    # 取得預測與實際座標
    y_test_pred_coordinates = np.array([label_to_coordinates[label] for label in y_test_pred_labels])
    y_test_actual_coordinates = np.array([label_to_coordinates[label] for label in y_test_actual])

    # 計算 MDE (Mean Distance Error)
    distances = np.linalg.norm(y_test_pred_coordinates - y_test_actual_coordinates, axis=1)
    mean_mde = np.mean(distances)

    # 記錄每個 RP 的 MDE
    mde_report_test = {}
    for true_label, distance in zip(y_test_actual, distances):
        if true_label not in mde_report_test:
            mde_report_test[true_label] = []
        mde_report_test[true_label].append(distance)

    # 計算測試資料的 MDE 平均值
    mde_report_test_avg = {label: {"mde": np.mean(dists), "count": len(dists)} for label, dists in mde_report_test.items()}

    # 儲存 MDE 結果到 JSON 檔案
    test_file_path = f"{root}/{modelname}.json"
    with open(test_file_path, "w") as f:
        json.dump(mde_report_test_avg, f, indent=4)

    # Needsave
    print(f"Test Data MDE report saved to: {test_file_path}")
    print(f"\nTest Data Mean MDE: {mean_mde:.4f} meters")
    ALL_mean_mde.append(mean_mde)


    base_model.save(f'{root}/DNN_best_model{ap}_{weekrepresent[i+1][0]}week_to_{weekrepresent[i+1][1]}week_{dataamount}dataPerRP.h5')


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 49)             │         3,185 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,595 (80.45 KB)

 Trainable params: 3,185 (12.44 KB)

 Non-trainable params: 17,408 (68.00 KB)

...

C:\Users\吳定洋\AppData\Local\Temp\ipykernel_20228\3920517792.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  test_processed_data = pd.concat([test_processed_data, group], ignore_index=True)


   AP1_Distance (mm)  AP2_Distance (mm)  AP3_Distance (mm)  AP4_Distance (mm)  \
0             4116.0             6723.0              168.0             9619.0   
1             3960.0             6753.0              164.0             9658.0   
2             4038.0             6694.0              -30.0             9541.0   
3             4116.0             6987.0              -30.0             9619.0   
4             3999.0             6606.0              164.0             9580.0   

   AP1_Rssi  AP2_Rssi  AP3_Rssi  AP4_Rssi  AP1_StdDev (mm)  AP2_StdDev (mm)  \
0     -61.0     -75.0     -48.0     -62.0            867.0            192.0   
1     -61.0     -75.0     -52.0     -66.0           2079.0            134.0   
2     -61.0     -75.0     -52.0     -67.0           2055.0            187.0   
3     -60.0     -75.0     -53.0     -65.0           2016.0            198.0   
4     -61.0     -76.0     -53.0     -67.0           2076.0             97.0   

   AP3_StdDev (mm)  AP4_StdDev (mm)  


,Training Set,Validation Set,Test Set
0,16,4,366
1,16,4,366
2,16,4,366
3,16,4,366
4,16,4,366
5,16,4,366
6,16,4,366
7,16,4,366
8,16,4,366
9,16,4,366


32
Epoch 1/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.1751 - loss: 9.9074 - val_accuracy: 0.1837 - val_loss: 8.1805
Epoch 2/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2398 - loss: 7.5666 - val_accuracy: 0.2551 - val_loss: 6.5010
Epoch 3/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2606 - loss: 5.9443 - val_accuracy: 0.3214 - val_loss: 5.4881
Epoch 4/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3419 - loss: 5.2348 - val_accuracy: 0.3673 - val_loss: 4.7462
Epoch 5/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4025 - loss: 4.2889 - val_accuracy: 0.4235 - val_loss: 4.1917
Epoch 6/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4520 - loss: 3.9746 - val_accuracy: 0.4643 - val_loss: 3.7616
Epoch 7/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4742 - loss: 3.3998 - val_accuracy: 0.5051 - val_loss: 3.3668
Epoch 8/10000
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4966 - loss: 3.3191 - val_a



Test Data Mean MDE: 0.0537 meters


In [12]:
print("training time")
print("[",end = '')
for t in ALL_trainingtime:
    print(f"{t:.2f}",end = ', ')
print("]")

print("Loss")
print("[",end = '')
for l in ALL_loss:
    print(f"{l:.4f}",end = ', ')
print("]")


print("Accuracy")
print("[",end = '')
for a in ALL_accuracy:
    print(f"{a:.4f}",end = ', ')
print("]")


print("MDE")
print("[",end = '')
for m in ALL_mean_mde:
    print(f"{m:.4f}",end = ', ')
print("]")

training time
[13.61, 25.91, 35.96, 13.92, 29.74, ]
Loss
[0.1282, 0.1368, 0.1347, 0.1995, 0.1939, ]
Accuracy
[0.9737, 0.9738, 0.9779, 0.9646, 0.9694, ]
MDE
[0.0335, 0.0361, 0.0391, 0.0511, 0.0537, ]
